# Sentimentos de tweets quando à Reforma da Previdência

In [1]:
library(twitteR)
library(ROAuth)
library(httr)
library(plyr)
library(stringr)
library(tidytext)
library(readr)
library(dplyr)





Attaching package: ‘plyr’

The following object is masked from ‘package:twitteR’:

    id


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:twitteR’:

    id, location

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
# parameters: api_key,api_secret,access_token,access_token_secret
setup_twitter_oauth('1', '2', 
                    '3', 
                    '4')

[1] "Using direct authentication"


In [3]:
tweetsrp <- searchTwitter('#reformaDaPrevidencia', n=5000, lang='pt')

Warning message in doRppAPICall("search/tweets", n, params = params, retryOnRateLimit = retryOnRateLimit, :
“5000 tweets were requested but the API can only return 1359”

In [4]:
head(tweetsrp)

[[1]]
[1] "GeraldoSiFi30: Ok, ok... a #Tuiuti foi o máximo neste #carnaval2018 ! Mas agora é vida real: dia 20 fev,  \" Vampirão\" vai colocar… https://t.co/On7tqxc8Rc"

[[2]]
[1] "LopesAnnamaria: RT @Fernanreira: Primeiro receba dos grandes devedores.\n#Reformadaprevidencia não. https://t.co/e1kdGqfTZz"

[[3]]
[1] "GilPereiraXavi1: RT @Fernanreira: Primeiro receba dos grandes devedores.\n#Reformadaprevidencia não. https://t.co/e1kdGqfTZz"

[[4]]
[1] "DiasDougfotos: @DeniseAlvesBR @Boatosorg BOA #ENTREVISTA DE #BOLSONARO para #RedeTV \n&gt; Não queremos dar carta branca para… https://t.co/LoEfbyK9on"

[[5]]
[1] "DiasDougfotos: @chilliperod @laurojardim BOA #ENTREVISTA DE #BOLSONARO para #RedeTV\n &gt; Não queremos dar carta branca para… https://t.co/JYe6MnlKlD"

[[6]]
[1] "cidcleybuarque: RT @tvcamara: O que pode mudar na aposentadoria dos funcionários públicos com a #ReformaDaPrevidência? Fizemos uma matéria especial sobre e…"


In [5]:
df <- twListToDF(tweetsrp)


In [6]:
head(df)

text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
"Ok, ok... a #Tuiuti foi o máximo neste #carnaval2018 ! Mas agora é vida real: dia 20 fev, "" Vampirão"" vai colocar… https://t.co/On7tqxc8Rc",FALSE,1,NA,2018-02-15 07:48:08,TRUE,NA,964043637683310592,NA,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",GeraldoSiFi30,0,FALSE,FALSE,NA,NA
RT @Fernanreira: Primeiro receba dos grandes devedores. #Reformadaprevidencia não. https://t.co/e1kdGqfTZz,FALSE,0,NA,2018-02-15 04:12:05,FALSE,NA,963989267058954240,NA,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",LopesAnnamaria,5,TRUE,FALSE,NA,NA
RT @Fernanreira: Primeiro receba dos grandes devedores. #Reformadaprevidencia não. https://t.co/e1kdGqfTZz,FALSE,0,NA,2018-02-15 02:59:38,FALSE,NA,963971033689673729,NA,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",GilPereiraXavi1,5,TRUE,FALSE,NA,NA
@DeniseAlvesBR @Boatosorg BOA #ENTREVISTA DE #BOLSONARO para #RedeTV &gt; Não queremos dar carta branca para… https://t.co/LoEfbyK9on,FALSE,1,DeniseAlvesBR,2018-02-15 01:59:26,TRUE,963941847981797376,963955880977985536,257140427,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",DiasDougfotos,0,FALSE,FALSE,NA,NA
@chilliperod @laurojardim BOA #ENTREVISTA DE #BOLSONARO para #RedeTV &gt; Não queremos dar carta branca para… https://t.co/JYe6MnlKlD,FALSE,2,chilliperod,2018-02-15 01:54:08,TRUE,963071264482119681,963954549978476544,248786695,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",DiasDougfotos,0,FALSE,FALSE,NA,NA
RT @tvcamara: O que pode mudar na aposentadoria dos funcionários públicos com a #ReformaDaPrevidência? Fizemos uma matéria especial sobre e…,FALSE,0,NA,2018-02-15 01:48:46,FALSE,NA,963953200104689664,NA,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",cidcleybuarque,9,TRUE,FALSE,NA,NA


In [7]:
oplexicon <- read_csv('oplexicon_v3.0/lexico_v3.0.txt', col_names = c('word', 'type', 'weight', 'other'), col_types = 
  cols(
    word = col_character(),
    type = col_character(),
    weight = col_integer(), 
    other = col_character()
  ))
head(oplexicon)

word,type,weight,other
=[,emot,-1,A
=@,emot,-1,A
=p,emot,-1,A
=P,emot,-1,A
=x,emot,-1,A
=d,emot,1,A


In [8]:
stopwords <- read_csv('portuguese-stopwords.txt', col_names = 'word')

Parsed with column specification:
cols(
  word = col_character()
)


In [9]:
tweets <- df %>%
    unnest_tokens(word,text) %>%
    anti_join(stopwords,by="word") 
head(tweets)

favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude,word
FALSE,1,NA,2018-02-15 07:48:08,TRUE,NA,964043637683310592,NA,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",GeraldoSiFi30,0,FALSE,FALSE,NA,NA,ok
FALSE,1,NA,2018-02-15 07:48:08,TRUE,NA,964043637683310592,NA,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",GeraldoSiFi30,0,FALSE,FALSE,NA,NA,ok
FALSE,1,NA,2018-02-15 07:48:08,TRUE,NA,964043637683310592,NA,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",GeraldoSiFi30,0,FALSE,FALSE,NA,NA,tuiuti
FALSE,1,NA,2018-02-15 07:48:08,TRUE,NA,964043637683310592,NA,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",GeraldoSiFi30,0,FALSE,FALSE,NA,NA,carnaval2018
FALSE,1,NA,2018-02-15 07:48:08,TRUE,NA,964043637683310592,NA,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",GeraldoSiFi30,0,FALSE,FALSE,NA,NA,vida
FALSE,1,NA,2018-02-15 07:48:08,TRUE,NA,964043637683310592,NA,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",GeraldoSiFi30,0,FALSE,FALSE,NA,NA,real


In [10]:
sentimentoTweets <- tweets %>%
    inner_join(oplexicon) %>%
    group_by(screenName) %>%
    summarize(peso = sum(weight, na.rm = TRUE))
head(sentimentoTweets)

Joining, by = "word"


screenName,peso
___DENISE___EU_,-1
_bielcelentano,0
_eitapa,0
_electriceel,-1
_guilhermeclapp,0
_hercules_k,0


In [11]:
sum(sentimentoTweets$peso)

[1] 185

In [12]:
mean(sentimentoTweets$peso)

[1] 0.2019651